In [17]:
file_path = "/Users/yoonhae/Downloads/crawling_processed_(데이터팀 검토) 그레인스캐너 패커 크롤링 정리 - Gulfood_정리포맷_20240123_134035.csv"
import pandas as pd

df = pd.read_csv(file_path)

In [2]:
import vertexai
from google.cloud import aiplatform
from langchain.llms.vertexai import VertexAI

PROJECT_ID = "grainscanner"  # @param {type:"string"}
REGION = "asia-northeast3"  # @param {type:"string"}

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

# Text model instance integrated with langChain
llm = VertexAI(
    model_name="text-bison", #"gemini-pro",#"text-bison",
    max_output_tokens=1024,
    temperature=0.4,
    top_p=0.8,
    top_k=40,
    verbose=True,
    location=REGION
)

target_column_list = ['homepage', 'extract_url', 'company_name', 'mail', 'phone', 'address', 'fax', 'products', 'error']

In [14]:
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json
import time

RETRY_COUNT = 3

def get_string_item(dict_item, key):
    if isinstance(dict_item, dict):
        value = dict_item.get(key, '')
        if not value:
            return ""
        
        while isinstance(value, list) and value:
            value = value[0]
        return value
    
    return ""

def get_list_item(dict_item, key):
    if isinstance(dict_item, dict):
        value = dict_item.get(key, [])
        return value
    elif isinstance(dict_item, list):
        return dict_item
    
    return []


def get_url_list_with_llm(llm, a_tag_list):
    json_parser = SimpleJsonOutputParser()

    filter_template = """
        당신은 유능하고 경험 많은 웹 제작자입니다.
        다음 global 회사 홈페이지에 태그된  a태그 리스트를 보고 
        회사에 contact 할 수 있는 정보를 담은 a태그와 
        취급 product를 설명하는 a태그만 골라서 href attribute를 추출해주세요.
        판단은 엄격하게 진행해주세요.

        Format instructions:
        ["href attribute", "href attribute"]

        -----------------------
        content :
        {a_tag_list}
    """

    filter_prompt_template = PromptTemplate(
        input_variables=["a_tag_list"], 
        template=filter_template, 
        output_parser=json_parser,
        partial_variables={
            "format_instructions": json_parser.get_format_instructions()
        }
    )

    chain = filter_prompt_template|llm|json_parser

    retry_count = 0
    last_error = None
    result_additional_filter_json = None
    while retry_count < RETRY_COUNT:
        try:
            result_additional_filter_json = chain.invoke({"a_tag_list": a_tag_list})
            break
        except Exception as e:
            print(f'    ** retry[{retry_count+1}] - ', e)
            last_error = str(e)
            time.sleep(1)
        retry_count += 1

    if result_additional_filter_json:
        print('   ', result_additional_filter_json)
        return result_additional_filter_json
    else:
        return {'error': last_error}

# 더불어 취급 products가 있다면 핵심 물품만 keyword list로 추출해주세요.
def get_firm_info_with_llm(llm, page_text):
    json_parser = SimpleJsonOutputParser()
    html_template = """
        당신은 유능한 global html parsor 입니다. 
        주어진 html 은 global 회사의 웹페이지입니다. 
        회사에 contact 할 수 있는 email, phone, fax, address 을 추출해주세요. 
        
        Format instructions:
        {format_instructions}

        ------------
        content:
        {page_text}
    """

    html_prompt_template = PromptTemplate(
        input_variables=["page_text"], 
        template=html_template, 
        output_parser=json_parser,
        partial_variables={
            "format_instructions": json_parser.get_format_instructions()# + "\n" + "json key are (email, phone, address, items)"
        }
    )

    html_chain = html_prompt_template | llm | json_parser
    retry_count = 0
    result_html_json = None
    last_error = None
    while retry_count < RETRY_COUNT:
        try :
            result_html_json = html_chain.invoke({"page_text": page_text})
            break
        except Exception as e:
            print(f'    ** retry[{retry_count+1}] - ', e)
            last_error = str(e)
            time.sleep(1)
        retry_count += 1

    if result_html_json:
        print('   ', '\n    '.join(json.dumps(result_html_json, indent=4).split('\n')))
        return result_html_json
    else:
        return {'error': last_error}

In [18]:
import requests
from bs4 import BeautifulSoup
from google.cloud import aiplatform
import time

from requests.adapters import HTTPAdapter, Retry

requests.adapters.DEFAULT_POOLSIZE = 100
retries = Retry(total=20)
# 세션을 생성하고 ConnectionPool 크기를 설정
session = requests.Session()
session.mount('https://', requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100, max_retries=retries))

from urllib.parse import urlparse

get_domain = lambda url: urlparse(url).netloc.replace('www.', '')

def change_protocol(url):
    # URL 파싱
    parsed_url = urlparse(url)
    # 프로토콜 추출
    protocol = parsed_url.scheme
    if protocol == 'http':
        return parsed_url._replace(scheme='https').geturl()
    elif protocol == 'https':
        return parsed_url._replace(scheme='http').geturl()

    return url

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

def extract_content(url, is_recursive=False):
    # 페이지 콘텐츠를 가져옵니다.

    try :
        time.sleep(0.5)
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            return response.status_code, soup
        elif response.status_code == 403:
            if not is_recursive:
                print(f'    ** page 조회 - retry')
                return extract_content(change_protocol(url), is_recursive=True)
            
        return response.status_code, None
    except Exception as e:
        return 500, str(e)


import pandas as pd

def to_dataframe(json_data):
    normalized_df = pd.json_normalize(json_data)
    total_column_list = list(normalized_df.columns)

    merged_df = pd.DataFrame(columns=target_column_list)

    for target_col in target_column_list:
        related_cols = [col for col in total_column_list if target_col in col]

        def merge_row(row):
            merged_data = []
            for item in row.dropna():
                if isinstance(item, dict):
                    merged_data.append(json.dumps(item, ensure_ascii=False))
                elif isinstance(item, list):
                    merged_data.extend([str(x) for x in item])
                elif not pd.isnull(item):
                    merged_data.append(item)
            if merged_data:
                return ';\n'.join(merged_data)
            else:
                return ''

        if related_cols:
            merged_df[target_col] = normalized_df[related_cols].apply(merge_row, axis=1)

    return merged_df


def is_string_not_none_nan(value):
    if pd.notna(value) and isinstance(value, str) and value:
        return True
    return False


def run(main_url):
    print(f'** [main page] 조회 - {main_url}')
    status, soup = extract_content(main_url)

    exists_email = False
    exists_phone = False

    result_df = pd.DataFrame(columns=target_column_list)
    if status == 200:
        print(f'  ** [llm] contact 정보 & products 추출')
        firm_info = {'homepage': main_url,
                     'extract_url': main_url}
        try :
            info = get_firm_info_with_llm(llm, soup.get_text().replace('\n\n\n', ''))
            if info:
                firm_info.update(info)
                sub_df = to_dataframe(firm_info)
                result_df = result_df.append(sub_df, ignore_index=True)
                exists_email = sub_df['mail'].apply(is_string_not_none_nan).any()
                exists_phone = sub_df['phone'].apply(is_string_not_none_nan).any()

                if exists_email and exists_phone:
                    return result_df
        except Exception as e:
            firm_info['error'] = str(e)

        print(f'  ** a tag 추출')
        a_list = soup.find_all('a', href=True)
        
        main_domain = get_domain(main_url)
        distinct_a_list = {}
        for link in a_list:
            if get_domain(link.get('href')).startswith(main_domain):
                distinct_a_list[link.get('href').strip('/')] = link
        
        if distinct_a_list:
            print(f'  ** [llm] 회사 정보와 관련있는 a tag 선별')
            page_list = get_url_list_with_llm(llm, [distinct_a_list.values()])
        
            for sub_url in page_list:
                print(f'  ** [sub page] 조회 - {sub_url}')
                sub_status, sub_soup = extract_content(sub_url)
                if sub_status == 200:
                    print(f'  ** [llm] contact 정보 & products 추출')
                    sub_firm_info = {'homepage': main_url,
                                    'extract_url': sub_url}
                    try :
                        info = get_firm_info_with_llm(llm, sub_soup.get_text().replace('\n\n\n', ''))
                        if info:
                            sub_firm_info.update(info)
                    except Exception as e:
                        sub_firm_info['error'] = str(e)
                    sub_df = to_dataframe(sub_firm_info)
                    result_df = result_df.append(sub_df, ignore_index=True)

                    exists_email = exists_email or sub_df['mail'].apply(is_string_not_none_nan).any()
                    exists_phone = exists_phone or sub_df['phone'].apply(is_string_not_none_nan).any()

                    if exists_email and exists_phone:
                        return result_df
                else:
                    print(f'    [{sub_status}] : 조회 실패')
        else:
            print(f'    [empty]')
    else:
        print(f'    [{status}] : 조회 실패')

    return result_df


In [23]:
# result = []
# for url in df['홈페이지 주소']:
#     if not pd.isnull(url) and not pd.isna(url):
#         sub_result = run(url)
#         if sub_result:
#             result.extend(sub_result)

result_total_df = pd.DataFrame(columns=target_column_list)
count = 0
for url in df['홈페이지 주소']:  # 45
    if not pd.isnull(url) and not pd.isna(url):        
        is_exists = (mail_df['homepage'] == url).any()
        if not is_exists:
            sub_result = run(url)
            if len(sub_result):
                result_total_df = result_total_df.append(sub_result, ignore_index=True)



** [main page] 조회 - https://deoleo.com/
  ** [llm] contact 정보 & products 추출
    {
        "contact": {
            "email": null,
            "phone": "0034 91 558 95 05",
            "fax": null,
            "address": "C/ Marie Curie 7\n4\u00ba plta.,\n28521 Rivas Vaciamadrid,\nMadrid (Espa\u00f1a)."
        }
    }
  ** a tag 추출
  ** [llm] 회사 정보와 관련있는 a tag 선별


/var/folders/wx/91r7nlnj64q2ql80lp6d9_kr0000gn/T/ipykernel_27402/424998281.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sub_df, ignore_index=True)


    ['https://deoleo.com/contacto/', 'https://deoleo.com/productos/']
  ** [sub page] 조회 - https://deoleo.com/contacto/
  ** [llm] contact 정보 & products 추출
    {
        "contact": {
            "email": null,
            "phone": "0034 91 558 95 05",
            "fax": null,
            "address": "C/ Marie Curie 7\n4\u00ba plta.,\n28521 Rivas Vaciamadrid,\nMadrid (Espa\u00f1a)."
        }
    }
  ** [sub page] 조회 - https://deoleo.com/productos/


/var/folders/wx/91r7nlnj64q2ql80lp6d9_kr0000gn/T/ipykernel_27402/424998281.py:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sub_df, ignore_index=True)


    [404] : 조회 실패
** [main page] 조회 - http://wingsagro.com


...

  ** [llm] contact 정보 & products 추출
    {
        "email": null,
        "phone": null,
        "fax": null,
        "address": null
    }
  ** a tag 추출
  ** [llm] 회사 정보와 관련있는 a tag 선별


/var/folders/wx/91r7nlnj64q2ql80lp6d9_kr0000gn/T/ipykernel_27402/424998281.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sub_df, ignore_index=True)


    ['https://www.zade.com.tr/bize-ulasin/', 'https://www.zade.com.tr/urunler/hindistan-cevizi-yagi/', 'https://www.zade.com.tr/urunler/aromali-zeytinyagi/', 'https://www.zade.com.tr/urunler/zeytinyagi/', 'https://www.zade.com.tr/urunler/aycicek-yaglari/', 'https://www.zade.com.tr/urunler/misir-yaglari/', 'https://www.zade.com.tr/urunler/fistik-yagi/', 'https://www.zade.com.tr/urunler/endustriyel-yag/', 'https://www.zade.com.tr/urunler/kanola-yagi/', 'https://www.zade.com.tr/urun/naturel-sizma-zeytinyagi-1lt-pet/', 'https://www.zade.com.tr/urun/naturel-sizma-zeytinyagi-2lt-pet/', 'https://www.zade.com.tr/urun/riviera-zeytinyagi-1-lt-pet/', 'https://www.zade.com.tr/urun/riviera-zeytinyagi-2-lt-pet/', 'https://www.zade.com.tr/urun/biberiye-aromali-naturel-sizma-zeytinyagi-250-ml-cam/', 'https://www.zade.com.tr/urun/limon-feslegen-aromali-naturel-sizma-zeytinyagi-250-ml-cam/', 'https://www.zade.com.tr/urun/nar-feslegen-limon-aromali-naturel-sizma-zeytinyagi-250-ml-cam/', 'https://www.zade

/var/folders/wx/91r7nlnj64q2ql80lp6d9_kr0000gn/T/ipykernel_27402/424998281.py:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(sub_df, ignore_index=True)
/var/folders/wx/91r7nlnj64q2ql80lp6d9_kr0000gn/T/ipykernel_27402/1293433820.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_total_df = result_total_df.append(sub_result, ignore_index=True)


    [500] : 조회 실패


In [ ]:
import pandas as pd

# 제공된 JSON 데이터
json_data = result
normalized_df = pd.json_normalize(json_data)

target_column_list = ['homepage', 'extract_url', 'company_name', 'mail', 'phone', 'address', 'fax', 'products', 'error']
total_column_list = list(normalized_df.columns)

merged_df = pd.DataFrame(columns=target_column_list)

for target_col in target_column_list:
    related_cols = [col for col in total_column_list if target_col in col]

    def merge_row(row):
        merged_data = []
        for item in row.dropna():
            if isinstance(item, dict):
                merged_data.append(json.dumps(item, ensure_ascii=False))
            elif isinstance(item, list):
                merged_data.extend([str(x) for x in item])
            elif not pd.isnull(item):
                merged_data.append(item)
        if merged_data:
            return ';\n'.join(merged_data)
        else:
            return ''

    if related_cols:
        merged_df[target_col] = normalized_df[related_cols].apply(merge_row, axis=1)

In [ ]:
import os
file_path = './output/beautifulsoup_sample.csv'
merged_df.to_csv(file_path, index=False)

# Numbers 애플리케이션으로 CSV 파일 열기
os.system(f'open -a Numbers {file_path}')

여러개 csv 하나로 합쳐서 확인하기

In [28]:
import pandas as pd

total_df = None
total = 0
for i in range(1, 6):
    file_path = f"./output/gulfood_crawling_20240124_00{i}.csv"
    print(file_path)
    partial_df = pd.read_csv(file_path)
    total += partial_df.groupby('homepage').count()

    if i == 1:
        total_df = partial_df
    else:
        total_df = total_df.append(partial_df, ignore_index=True)

./output/gulfood_crawling_20240124_001.csv
./output/gulfood_crawling_20240124_002.csv
./output/gulfood_crawling_20240124_003.csv
./output/gulfood_crawling_20240124_004.csv
./output/gulfood_crawling_20240124_005.csv


/var/folders/wx/91r7nlnj64q2ql80lp6d9_kr0000gn/T/ipykernel_27402/2973475628.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = total_df.append(partial_df, ignore_index=True)
/var/folders/wx/91r7nlnj64q2ql80lp6d9_kr0000gn/T/ipykernel_27402/2973475628.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = total_df.append(partial_df, ignore_index=True)
/var/folders/wx/91r7nlnj64q2ql80lp6d9_kr0000gn/T/ipykernel_27402/2973475628.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = total_df.append(partial_df, ignore_index=True)
/var/folders/wx/91r7nlnj64q2ql80lp6d9_kr0000gn/T/ipykernel_27402/2973475628.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a

In [29]:
def unique_list(series):
    # NaN 값을 제거하고, 중복을 제거한 후 리스트로 변환
    return series.dropna().drop_duplicates().tolist()


# 'homepage'를 기준으로 그룹화하고 각 컬럼에 unique_list 함수 적용
grouped_df = total_df.groupby('homepage').agg(unique_list).reset_index()


In [ ]:
def convert_to_string(value):
    # 값이 리스트인 경우
    if isinstance(value, list):
        # 빈 리스트인 경우 None 반환
        if not value:
            return ''
        # 리스트의 요소들을 문자열로 결합
        return ";\n".join(str(item) for item in value)
    # 리스트가 아닌 경우, 값을 그대로 반환
    return value

# 'homepage' 컬럼을 제외한 모든 컬럼에 대해 함수 적용
for col in grouped_df.columns:
    if col != 'homepage':
        grouped_df[col] = grouped_df[col].apply(convert_to_string)

print(grouped_df)


In [30]:
import pandas as pd

# 가정: df는 이미 생성된 DataFrame이며 'mail' 컬럼을 포함하고 있음

# 'mail' 컬럼이 빈 리스트인 행만 필터링
mail_df = grouped_df[grouped_df['mail'].apply(lambda x: len(x) > 0)]



In [ ]:
grouped_df.to_csv('./output/gulfood_20240123_1645.csv')